In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from tqdm import tqdm
from imblearn.under_sampling import RandomUnderSampler
from spellchecker import SpellChecker
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import FrenchStemmer
from unidecode import unidecode
import json
import random
import ast
from sklearn.metrics import mean_squared_error, mean_absolute_error
from transformers import pipeline
from rank_bm25 import BM25Okapi
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
import pickle
import numpy as np
import warnings

C:\Users\courn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Ignore warnings
warnings.filterwarnings("ignore")
warnings.warn("DelftStack")
warnings.warn("Do not show this message")
print("No Warning Shown")

No Warning Shown


# Scrapping

In [2]:
# get all compagny url
page = 1
result_len = 0
compagnies_url = []
while True:
    url = f"https://fr.trustpilot.com/categories/car_dealer?page={page}"
    response = requests.get(url)
    web_page = response.text
    soup = BeautifulSoup(web_page, "html.parser")

    resp = soup.select("a[data-business-unit-card-link]")
    for res in resp:
        url = res["href"].replace("/review/", "")
        compagnies_url.append(url)
    if result_len == 0:
        result_len = len(resp)
    elif result_len != len(resp):
        break
    print(page)
    page += 1

1
2


KeyboardInterrupt: 

In [3]:
def compagny_url(compagny_url):
    # Initialize lists
    review_titles = []
    review_ratings = []
    review_texts = []
    review_locations = []
    page_number = []

    # Set Trustpilot page numbers to scrape here
    from_page = 1
    to_page = 500

    for i in range(from_page, to_page + 1):
        response = requests.get(f"https://fr.trustpilot.com/review/{compagny_url}?page={i}")
        web_page = response.text
        soup = BeautifulSoup(web_page, "html.parser")

        if soup.find_all("article") == []:
            break

        for review in soup.find_all("article"):
            # Review titles
            review_title = review.select_one("a[data-review-title-typography]")
            if review_title == None:
                review_titles.append("")
            else:
                review_titles.append(review_title.getText())

            # Review text
            review_text = review.select_one("p[data-service-review-text-typography]")
            if review_text == None:
                review_texts.append("")
            else:
                review_texts.append(review_text.getText())

            # Review ratings
            review_rating = review.select_one("div[data-service-review-rating]")
            review_ratings.append(review_rating["data-service-review-rating"])
                
            review_location = review.select_one("div[data-consumer-country-typography]")
            if review_location == None:
                review_locations.append("")
            else:
                review_locations.append(review_location.getText())
            # Trustpilot page number
            page_number.append(i)

    # Create final dataframe from lists
    return pd.DataFrame(list(zip([compagny_url for i in range(len(review_titles))], review_titles, review_ratings, review_texts, review_locations)),
                    columns =['compagny', 'review_title', 'review_rating', 'review_text', 'review_location'])

In [4]:
df = pd.DataFrame()
for compagny in tqdm(compagnies_url):
    result = compagny_url(compagny)
    df = pd.concat([df, result])

df.to_csv("reviews.csv", index=False)

  0%|          | 0/79 [00:00<?, ?it/s]

100%|██████████| 79/79 [09:13<00:00,  7.01s/it] 


# Init functions

In [4]:
try:
    useless_words = pd.read_csv("most_frequent_words_mixed.csv", header=None)[0].tolist()[:100]
except:
    useless_words = []

In [69]:
STEMMER = FrenchStemmer()
spell = SpellChecker(language='fr')
pipe = pipeline("text-classification", model="tblard/tf-allocine")

def preprocess_text(text):
    # Suppression des accents
    text = unidecode(text)
    # Suppression du code HTML
    text = re.sub(re.compile("<.*?>"), "", text)
    text = re.sub(r'[^a-zA-Z0-9/s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    # Suppresssion des nombres
    text = re.sub(r'[0-9]+', ' ', text)
    # Supprimer les lignes vides
    text = text.split('\n')
    text = [line.strip() for line in text if len(line) > 0]
    text = ' '.join(text)
    # Supprimer les liens
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    # Lemmatiser les mots
    tokens = word_tokenize(text.lower(), language='french')
    return tokens

n_grams = lambda tokens, n: [" ".join(tokens[i:i+n]) for i in range(len(tokens)-n+1)]

def text2Token(text, spelling = True, stem = True, len_word_min = 2, spell = spell, useless_words = useless_words, use_n_grams = True):
    stopword = stopwords.words('french')
    word_tokens = preprocess_text(text)
    word_tokens = [word for word in word_tokens if word not in stopword and word not in useless_words and len(word) > len_word_min]
    if spelling:
        word_tokens = [spell.correction(word) for word in word_tokens]
        word_tokens = [word for word in word_tokens if word != None]
    if stem:
        word_tokens = [STEMMER.stem(token) for token in word_tokens]
    if use_n_grams:
        word_tokens = word_tokens + n_grams(word_tokens, 2) + n_grams(word_tokens, 3)
    return word_tokens

def getMostFrequentWords(documents, top=10):
    # Compter les mots
    word_count = {}
    for doc in documents:
        for word in doc:
            if word in word_count:
                word_count[word] += 1
            else:
                word_count[word] = 1

    # Trier les mots par fréquence décroissante
    word_count = sorted(word_count.items(), key=lambda x: x[1], reverse=True)

    if top == float('inf'):
        return word_count
    
    return word_count[:top]

All model checkpoint layers were used when initializing TFCamembertForSequenceClassification.

All the layers of TFCamembertForSequenceClassification were initialized from the model checkpoint at tblard/tf-allocine.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFCamembertForSequenceClassification for predictions without further training.


# Data Cleaning

In [4]:
# Preprocess data
print("Preprocess data")
df = pd.read_csv("reviews.csv")
df = df.dropna(subset=['review_text'])
df = df[df["review_location"]  == "FR"]
df = df.drop_duplicates(subset=['review_text'])
df = df.reset_index(drop=True)
df = df[['review_text', 'review_rating']]

Preprocess data


## Get most frequent words list

In [5]:
df_prepared_data = []
for i in tqdm(range(len(df))):
    df_prepared_data.append(text2Token(df.at[i, "review_text"], stem=False, useless_words=[]))

  0%|          | 0/16933 [00:00<?, ?it/s]

  0%|          | 19/16933 [00:09<2:19:33,  2.02it/s]


KeyboardInterrupt: 

In [10]:
most_frequent_words = getMostFrequentWords(df_prepared_data, top=1000)
pd.DataFrame(most_frequent_words, columns=["word", "count"]).to_csv("most_frequent_words.csv", index=False)

## Traslate most frequent words list to english (for better performance of the pipe)

In [2]:
most_frequent_words = pd.read_csv("most_frequent_words.csv")

In [91]:
if "en" not in most_frequent_words.columns:
    most_frequent_words["en"] = [None for i in range(len(most_frequent_words))]

In [107]:
import translators as ts

def translate(word):
    try:
        return ts.tencent(word, to_language="en", from_language="fr")
    except Exception as e:
        return None

In [108]:
for i in tqdm(range(len(most_frequent_words))):
    if most_frequent_words.at[i, "en"] == None:
        most_frequent_words.at[i, "en"] = translate(most_frequent_words.at[i, "word"])

100%|██████████| 1000/1000 [47:31<00:00,  2.85s/it] 


In [109]:
most_frequent_words.to_csv("most_frequent_words.csv", index=False)

## Get the sentiment of the words

In [75]:
most_frequent_words = pd.read_csv("most_frequent_words.csv")

In [8]:
if "sentiment" in most_frequent_words.columns:
    most_frequent_words["sentiment"] = [None for i in range(len(most_frequent_words))]

In [24]:
for i in tqdm(range(len(most_frequent_words))):
    if most_frequent_words.at[i, "sentiment"] == None and str(most_frequent_words.at[i, "word"]) != "nan":
        pipe_res = pipe(most_frequent_words.at[i, "word"])[0]
        most_frequent_words.at[i, "sentiment"] = pipe_res["label"] if pipe_res["score"] > 0.65 else "MIXED"

100%|██████████| 1000/1000 [04:29<00:00,  3.71it/s]


In [25]:
most_frequent_words.to_csv("most_frequent_words.csv", index=False)

In [27]:
most_frequent_words[most_frequent_words["sentiment"] == "MIXED"][["word"]].to_csv("most_frequent_words_mixed.csv", index=False, header=False)

In [3]:
useless_words = most_frequent_words[most_frequent_words["sentiment"] == "MIXED"]["word"].tolist()[100:]

## Generate train/test

In [31]:
# Generate train tokenized dataset
print("Generate train tokenized dataset")
train = df.copy()
rus = RandomUnderSampler(random_state=0)
X = train[["review_text"]]
y = train["review_rating"]
X_resampled, y_resampled = rus.fit_resample(X, y)
train = pd.concat([X_resampled, y_resampled], axis=1)
train_idx = train.index.tolist()
train = train.reset_index(drop=True)
train.to_csv("train_untokenized.csv", index=False)

# Spell check for train dataset
print("Spell check for train dataset")
for i in tqdm(range(len(train))):
    train.at[i, "review_text"] = json.dumps(text2Token(train.at[i, "review_text"]))

# Save train dataset
print("Save train dataset")
train.to_csv("reviews_train.csv", index=False)

# Generate test dataset
print("Generate test dataset")
test = df.copy()
test = test.drop(train_idx)
rus = RandomUnderSampler(random_state=0)
X = test[["review_text"]]
y = test["review_rating"]
X_resampled, y_resampled = rus.fit_resample(X, y)
test = pd.concat([X_resampled, y_resampled], axis=1)

# Save test dataset
print("Save test dataset")
test.to_csv("reviews_test.csv", index=False)

Generate train dataset
Spell check for train dataset


  0%|          | 0/1905 [00:00<?, ?it/s]

100%|██████████| 1905/1905 [17:02<00:00,  1.86it/s] 

Save train dataset
Generate test dataset
Save test dataset


# Improve pipe

In [4]:
train_untokenized = pd.read_csv("train_untokenized.csv")

In [37]:
def get_pipe_res(text):
    try:
        return pipe(text)[0]
    except:
        return None

pipe_score = []
for i in tqdm(range(len(train_untokenized))):
    pipe_score.append(get_pipe_res(train_untokenized.at[i, "review_text"]))

real_score = train_untokenized["review_rating"].tolist()

while None in pipe_score:
    idx = pipe_score.index(None)
    del real_score[idx]
    del pipe_score[idx]

pipe_label = [elmt["label"] for elmt in pipe_score]
pipe_score = [elmt["score"] for elmt in pipe_score]
df_pipe = pd.DataFrame(list(zip(real_score, pipe_label, pipe_score)), columns=["real_score", "pipe_label", "pipe_score"])
df_pipe["pipe_label"] = df_pipe["pipe_label"].apply(lambda x: 1 if x == "POSITIVE" else 0)

100%|██████████| 1990/1990 [11:14<00:00,  2.95it/s]


In [39]:
pipe_score = []
for i in tqdm(range(len(test))):
    pipe_score.append(get_pipe_res(test.at[i, "review_text"]))

real_score = test["review_rating"].tolist()

while None in pipe_score:
    idx = pipe_score.index(None)
    del real_score[idx]
    del pipe_score[idx]

pipe_label = [elmt["label"] for elmt in pipe_score]
pipe_score = [elmt["score"] for elmt in pipe_score]
df_pipe_test = pd.DataFrame(list(zip(real_score, pipe_label, pipe_score)), columns=["real_score", "pipe_label", "pipe_score"])
df_pipe_test["pipe_label"] = df_pipe_test["pipe_label"].apply(lambda x: 1 if x == "POSITIVE" else 0)

100%|██████████| 608/608 [04:27<00:00,  2.27it/s]


In [41]:

reg = LinearRegression().fit(df_pipe[["pipe_score", "pipe_label"]], df_pipe["real_score"])

y_pred = reg.predict(df_pipe_test[["pipe_score", "pipe_label"]])

print(f"MAE: {mean_absolute_error(df_pipe_test['real_score'], y_pred)}")
print(f"MSE: {mean_squared_error(df_pipe_test['real_score'], y_pred)}")

MAE: 0.9053851431359945
MSE: 1.14167469575748


In [42]:
reg = LogisticRegression().fit(df_pipe[["pipe_score", "pipe_label"]], df_pipe["real_score"])

y_pred = reg.predict(df_pipe_test[["pipe_score", "pipe_label"]])

print(f"MAE: {mean_absolute_error(df_pipe_test['real_score'], y_pred)}")
print(f"MSE: {mean_squared_error(df_pipe_test['real_score'], y_pred)}")

MAE: 0.7337770382695508
MSE: 1.276206322795341


In [43]:
reg = SVR().fit(df_pipe[["pipe_score", "pipe_label"]], df_pipe["real_score"])

y_pred = reg.predict(df_pipe_test[["pipe_score", "pipe_label"]])

print(f"MAE: {mean_absolute_error(df_pipe_test['real_score'], y_pred)}")
print(f"MSE: {mean_squared_error(df_pipe_test['real_score'], y_pred)}")

MAE: 0.8012523209233274
MSE: 0.9530500843969638


In [44]:
reg = XGBRegressor().fit(df_pipe[["pipe_score", "pipe_label"]], df_pipe["real_score"])

y_pred = reg.predict(df_pipe_test[["pipe_score", "pipe_label"]])

print(f"MAE: {mean_absolute_error(df_pipe_test['real_score'], y_pred)}")
print(f"MSE: {mean_squared_error(df_pipe_test['real_score'], y_pred)}")

MAE: 0.7935105391627738
MSE: 0.9682374371766631


In [86]:
reg = RandomForestRegressor().fit(df_pipe[["pipe_score", "pipe_label"]], df_pipe["real_score"])

y_pred = reg.predict(df_pipe_test[["pipe_score", "pipe_label"]])

print(f"MAE: {mean_absolute_error(df_pipe_test['real_score'], y_pred)}")
print(f"MSE: {mean_squared_error(df_pipe_test['real_score'], y_pred)}")

MAE: 0.6967306077172966
MSE: 0.8983029138491782


In [87]:
pickle.dump(reg, open("pipe_model.pkl", "wb"))

# Improve bm25

In [87]:
def getTopDocs(bm25, query, documents, ratings, top=5):
    # Calculer les scores de similarité
    scores = bm25.get_scores(query)

    # Associer chaque avis à son score
    doc_scores = list(zip(documents, scores, ratings))

    # Trier les avis par score décroissant
    return sorted(doc_scores, key=lambda x: x[1], reverse=True)[:top]

In [88]:
train = pd.read_csv("reviews_train.csv")
train["review_text"] = train["review_text"].apply(lambda x: ast.literal_eval(x))
documents = train["review_text"].tolist()
ratings = train["review_rating"].tolist()
bm25 = BM25Okapi(documents)

bm25_scores_1, bm25_scores_2, bm25_scores_3, bm25_scores_4, bm25_scores_5 = [], [], [], [], []
bm25_ratings_1, bm25_ratings_2, bm25_ratings_3, bm25_ratings_4, bm25_ratings_5 = [], [], [], [], []
for i in tqdm(range(len(train))):
    top_docs = getTopDocs(bm25, train["review_text"].iloc[i], documents, ratings, top=6)[1:]
    bm25_scores_1.append(top_docs[0][1])
    bm25_scores_2.append(top_docs[1][1])
    bm25_scores_3.append(top_docs[2][1])
    bm25_scores_4.append(top_docs[3][1])
    bm25_scores_5.append(top_docs[4][1])
    bm25_ratings_1.append(top_docs[0][2])
    bm25_ratings_2.append(top_docs[1][2])
    bm25_ratings_3.append(top_docs[2][2])
    bm25_ratings_4.append(top_docs[3][2])
    bm25_ratings_5.append(top_docs[4][2])

real_score = train["review_rating"].tolist()

df_bm25 = pd.DataFrame(list(zip(real_score, bm25_ratings_1, bm25_ratings_2, bm25_ratings_3, bm25_ratings_4, bm25_ratings_5, bm25_scores_1, bm25_scores_2, bm25_scores_3, bm25_scores_4, bm25_scores_5)), columns=["real_score", "bm25_ratings_1", "bm25_ratings_2", "bm25_ratings_3", "bm25_ratings_4", "bm25_ratings_5", "bm25_scores_1", "bm25_scores_2", "bm25_scores_3", "bm25_scores_4", "bm25_scores_5"])

100%|██████████| 1905/1905 [00:40<00:00, 47.46it/s] 


In [89]:
test = pd.read_csv("reviews_test.csv")

bm25_scores_1, bm25_scores_2, bm25_scores_3, bm25_scores_4, bm25_scores_5 = [], [], [], [], []
bm25_ratings_1, bm25_ratings_2, bm25_ratings_3, bm25_ratings_4, bm25_ratings_5 = [], [], [], [], []
for i in tqdm(range(len(test))):
    top_docs = getTopDocs(bm25, text2Token(test["review_text"].iloc[i]), documents, ratings, top=5)
    bm25_scores_1.append(top_docs[0][1])
    bm25_scores_2.append(top_docs[1][1])
    bm25_scores_3.append(top_docs[2][1])
    bm25_scores_4.append(top_docs[3][1])
    bm25_scores_5.append(top_docs[4][1])
    bm25_ratings_1.append(top_docs[0][2])
    bm25_ratings_2.append(top_docs[1][2])
    bm25_ratings_3.append(top_docs[2][2])
    bm25_ratings_4.append(top_docs[3][2])
    bm25_ratings_5.append(top_docs[4][2])

real_score = test["review_rating"].tolist()

df_bm25_test = pd.DataFrame(list(zip(real_score, bm25_ratings_1, bm25_ratings_2, bm25_ratings_3, bm25_ratings_4, bm25_ratings_5, bm25_scores_1, bm25_scores_2, bm25_scores_3, bm25_scores_4, bm25_scores_5)), columns=["real_score", "bm25_ratings_1", "bm25_ratings_2", "bm25_ratings_3", "bm25_ratings_4", "bm25_ratings_5", "bm25_scores_1", "bm25_scores_2", "bm25_scores_3", "bm25_scores_4", "bm25_scores_5"])

100%|██████████| 608/608 [03:29<00:00,  2.91it/s]


In [92]:
reg = LinearRegression().fit(df_bm25[["bm25_ratings_1", "bm25_ratings_2", "bm25_ratings_3", "bm25_ratings_4", "bm25_ratings_5", "bm25_scores_1", "bm25_scores_2", "bm25_scores_3", "bm25_scores_4", "bm25_scores_5"]], df_bm25["real_score"])

y_pred = reg.predict(df_bm25_test[["bm25_ratings_1", "bm25_ratings_2", "bm25_ratings_3", "bm25_ratings_4", "bm25_ratings_5", "bm25_scores_1", "bm25_scores_2", "bm25_scores_3", "bm25_scores_4", "bm25_scores_5"]])

print(f"MAE: {mean_absolute_error(df_bm25_test['real_score'], y_pred)}")
print(f"MSE: {mean_squared_error(df_bm25_test['real_score'], y_pred)}")

MAE: 0.8897406237777181
MSE: 1.2068894929259424


In [93]:
pickle.dump(reg, open("bm25_model.pkl", "wb"))

In [80]:
reg = LogisticRegression().fit(df_bm25[["bm25_ratings_1", "bm25_ratings_2", "bm25_ratings_3", "bm25_ratings_4", "bm25_ratings_5", "bm25_scores_1", "bm25_scores_2", "bm25_scores_3", "bm25_scores_4", "bm25_scores_5"]], df_bm25["real_score"])

y_pred = reg.predict(df_bm25_test[["bm25_ratings_1", "bm25_ratings_2", "bm25_ratings_3", "bm25_ratings_4", "bm25_ratings_5", "bm25_scores_1", "bm25_scores_2", "bm25_scores_3", "bm25_scores_4", "bm25_scores_5"]])

print(f"MAE: {mean_absolute_error(df_bm25_test['real_score'], y_pred)}")
print(f"MSE: {mean_squared_error(df_bm25_test['real_score'], y_pred)}")

MAE: 0.9638157894736842
MSE: 1.8651315789473684


C:\Users\courn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [81]:
reg = SVR().fit(df_bm25[["bm25_ratings_1", "bm25_ratings_2", "bm25_ratings_3", "bm25_ratings_4", "bm25_ratings_5", "bm25_scores_1", "bm25_scores_2", "bm25_scores_3", "bm25_scores_4", "bm25_scores_5"]], df_bm25["real_score"])

y_pred = reg.predict(df_bm25_test[["bm25_ratings_1", "bm25_ratings_2", "bm25_ratings_3", "bm25_ratings_4", "bm25_ratings_5", "bm25_scores_1", "bm25_scores_2", "bm25_scores_3", "bm25_scores_4", "bm25_scores_5"]])

print(f"MAE: {mean_absolute_error(df_bm25_test['real_score'], y_pred)}")
print(f"MSE: {mean_squared_error(df_bm25_test['real_score'], y_pred)}")

MAE: 0.8957257740097123
MSE: 1.2168132659887394


In [82]:
reg = XGBRegressor().fit(df_bm25[["bm25_ratings_1", "bm25_ratings_2", "bm25_ratings_3", "bm25_ratings_4", "bm25_ratings_5", "bm25_scores_1", "bm25_scores_2", "bm25_scores_3", "bm25_scores_4", "bm25_scores_5"]], df_bm25["real_score"])

y_pred = reg.predict(df_bm25_test[["bm25_ratings_1", "bm25_ratings_2", "bm25_ratings_3", "bm25_ratings_4", "bm25_ratings_5", "bm25_scores_1", "bm25_scores_2", "bm25_scores_3", "bm25_scores_4", "bm25_scores_5"]])

print(f"MAE: {mean_absolute_error(df_bm25_test['real_score'], y_pred)}")
print(f"MSE: {mean_squared_error(df_bm25_test['real_score'], y_pred)}")

MAE: 0.9585107639431953
MSE: 1.3620632070876018


In [83]:
reg = RandomForestRegressor().fit(df_bm25[["bm25_ratings_1", "bm25_ratings_2", "bm25_ratings_3", "bm25_ratings_4", "bm25_ratings_5", "bm25_scores_1", "bm25_scores_2", "bm25_scores_3", "bm25_scores_4", "bm25_scores_5"]], df_bm25["real_score"])

y_pred = reg.predict(df_bm25_test[["bm25_ratings_1", "bm25_ratings_2", "bm25_ratings_3", "bm25_ratings_4", "bm25_ratings_5", "bm25_scores_1", "bm25_scores_2", "bm25_scores_3", "bm25_scores_4", "bm25_scores_5"]])

print(f"MAE: {mean_absolute_error(df_bm25_test['real_score'], y_pred)}")
print(f"MSE: {mean_squared_error(df_bm25_test['real_score'], y_pred)}")

MAE: 0.9271417214912281
MSE: 1.2284654359009504


# Improve BM25 + Pipe

In [19]:
def getTopDocs(bm25, query, documents, ratings, top=5):
    # Calculer les scores de similarité
    scores = bm25.get_scores(query)

    # Associer chaque avis à son score
    doc_scores = list(zip(documents, scores, ratings))

    # Trier les avis par score décroissant
    return sorted(doc_scores, key=lambda x: x[1], reverse=True)[:top]

In [20]:
train_untokenized = pd.read_csv("train_untokenized.csv")

def get_pipe_res(text):
    try:
        return pipe(text)[0]
    except:
        return None

pipe_score = []
for i in tqdm(range(len(train_untokenized))):
    pipe_score.append(get_pipe_res(train_untokenized.at[i, "review_text"]))

real_score = train_untokenized["review_rating"].tolist()

review_text_tokenized = []
for i in tqdm(range(len(train_untokenized))):
    if str(train_untokenized.at[i, "review_text"]) == "nan":
        review_text_tokenized.append(["None"])
        continue
    review_text_tokenized.append(text2Token(train_untokenized.at[i, "review_text"]))

train_untokenized["review_text_tokenized"] = review_text_tokenized

documents = train_untokenized["review_text_tokenized"].tolist()
ratings = train_untokenized["review_rating"].tolist()
bm25 = BM25Okapi(documents)

bm25_scores_1, bm25_scores_2, bm25_scores_3, bm25_scores_4, bm25_scores_5 = [], [], [], [], []
bm25_ratings_1, bm25_ratings_2, bm25_ratings_3, bm25_ratings_4, bm25_ratings_5 = [], [], [], [], []
for i in tqdm(range(len(train_untokenized))):
    top_docs = getTopDocs(bm25, train_untokenized["review_text_tokenized"].iloc[i], documents, ratings, top=6)[1:]
    bm25_scores_1.append(top_docs[0][1])
    bm25_scores_2.append(top_docs[1][1])
    bm25_scores_3.append(top_docs[2][1])
    bm25_scores_4.append(top_docs[3][1])
    bm25_scores_5.append(top_docs[4][1])
    bm25_ratings_1.append(top_docs[0][2])
    bm25_ratings_2.append(top_docs[1][2])
    bm25_ratings_3.append(top_docs[2][2])
    bm25_ratings_4.append(top_docs[3][2])
    bm25_ratings_5.append(top_docs[4][2])

while None in pipe_score:
    idx = pipe_score.index(None)
    del real_score[idx]
    del pipe_score[idx]
    del bm25_scores_1[idx]
    del bm25_scores_2[idx]
    del bm25_scores_3[idx]
    del bm25_scores_4[idx]
    del bm25_scores_5[idx]
    del bm25_ratings_1[idx]
    del bm25_ratings_2[idx]
    del bm25_ratings_3[idx]
    del bm25_ratings_4[idx]
    del bm25_ratings_5[idx]

pipe_label = [elmt["label"] for elmt in pipe_score]
pipe_score = [elmt["score"] for elmt in pipe_score]
df_both = pd.DataFrame(list(zip(real_score, pipe_label, pipe_score, bm25_ratings_1, bm25_ratings_2, bm25_ratings_3, bm25_ratings_4, bm25_ratings_5, bm25_scores_1, bm25_scores_2, bm25_scores_3, bm25_scores_4, bm25_scores_5)), columns=["real_score", "pipe_label", "pipe_score", "bm25_ratings_1", "bm25_ratings_2", "bm25_ratings_3", "bm25_ratings_4", "bm25_ratings_5", "bm25_scores_1", "bm25_scores_2", "bm25_scores_3", "bm25_scores_4", "bm25_scores_5"])
df_both["pipe_label"] = df_both["pipe_label"].apply(lambda x: 1 if x == "POSITIVE" else 0)

100%|██████████| 1990/1990 [00:41<00:00, 47.90it/s] 


In [21]:
test = pd.read_csv("reviews_test.csv")

pipe_score = []
for i in tqdm(range(len(test))):
    pipe_score.append(get_pipe_res(test.at[i, "review_text"]))

real_score = test["review_rating"].tolist()

review_text_tokenized = []
for i in tqdm(range(len(test))):
    if str(test.at[i, "review_text"]) == "nan":
        review_text_tokenized.append(["None"])
        continue
    review_text_tokenized.append(text2Token(test.at[i, "review_text"]))

test["review_text_tokenized"] = review_text_tokenized

bm25_scores_1, bm25_scores_2, bm25_scores_3, bm25_scores_4, bm25_scores_5 = [], [], [], [], []
bm25_ratings_1, bm25_ratings_2, bm25_ratings_3, bm25_ratings_4, bm25_ratings_5 = [], [], [], [], []

for i in tqdm(range(len(test))):
    top_docs = getTopDocs(bm25, test["review_text_tokenized"].iloc[i], documents, ratings, top=5)
    bm25_scores_1.append(top_docs[0][1])
    bm25_scores_2.append(top_docs[1][1])
    bm25_scores_3.append(top_docs[2][1])
    bm25_scores_4.append(top_docs[3][1])
    bm25_scores_5.append(top_docs[4][1])
    bm25_ratings_1.append(top_docs[0][2])
    bm25_ratings_2.append(top_docs[1][2])
    bm25_ratings_3.append(top_docs[2][2])
    bm25_ratings_4.append(top_docs[3][2])
    bm25_ratings_5.append(top_docs[4][2])

while None in pipe_score:
    idx = pipe_score.index(None)
    del real_score[idx]
    del pipe_score[idx]
    del bm25_scores_1[idx]
    del bm25_scores_2[idx]
    del bm25_scores_3[idx]
    del bm25_scores_4[idx]
    del bm25_scores_5[idx]
    del bm25_ratings_1[idx]
    del bm25_ratings_2[idx]
    del bm25_ratings_3[idx]
    del bm25_ratings_4[idx]
    del bm25_ratings_5[idx]

pipe_label = [elmt["label"] for elmt in pipe_score]
pipe_score = [elmt["score"] for elmt in pipe_score]
df_both_test = pd.DataFrame(list(zip(real_score, pipe_label, pipe_score, bm25_ratings_1, bm25_ratings_2, bm25_ratings_3, bm25_ratings_4, bm25_ratings_5, bm25_scores_1, bm25_scores_2, bm25_scores_3, bm25_scores_4, bm25_scores_5)), columns=["real_score", "pipe_label", "pipe_score", "bm25_ratings_1", "bm25_ratings_2", "bm25_ratings_3", "bm25_ratings_4", "bm25_ratings_5", "bm25_scores_1", "bm25_scores_2", "bm25_scores_3", "bm25_scores_4", "bm25_scores_5"])
df_both_test["pipe_label"] = df_both_test["pipe_label"].apply(lambda x: 1 if x == "POSITIVE" else 0)

100%|██████████| 608/608 [00:11<00:00, 50.84it/s] 


In [24]:
reg = LinearRegression().fit(df_both[["pipe_score", "pipe_label", "bm25_ratings_1", "bm25_ratings_2", "bm25_ratings_3", "bm25_ratings_4", "bm25_ratings_5", "bm25_scores_1", "bm25_scores_2", "bm25_scores_3", "bm25_scores_4", "bm25_scores_5"]], df_both["real_score"])

y_pred = reg.predict(df_both_test[["pipe_score", "pipe_label", "bm25_ratings_1", "bm25_ratings_2", "bm25_ratings_3", "bm25_ratings_4", "bm25_ratings_5", "bm25_scores_1", "bm25_scores_2", "bm25_scores_3", "bm25_scores_4", "bm25_scores_5"]])

print(f"MAE: {mean_absolute_error(df_both_test['real_score'], y_pred)}")
print(f"MSE: {mean_squared_error(df_both_test['real_score'], y_pred)}")

MAE: 0.760856239534132
MSE: 0.8547959146309988


In [25]:
reg = LogisticRegression().fit(df_both[["pipe_score", "pipe_label", "bm25_ratings_1", "bm25_ratings_2", "bm25_ratings_3", "bm25_ratings_4", "bm25_ratings_5", "bm25_scores_1", "bm25_scores_2", "bm25_scores_3", "bm25_scores_4", "bm25_scores_5"]], df_both["real_score"])

y_pred = reg.predict(df_both_test[["pipe_score", "pipe_label", "bm25_ratings_1", "bm25_ratings_2", "bm25_ratings_3", "bm25_ratings_4", "bm25_ratings_5", "bm25_scores_1", "bm25_scores_2", "bm25_scores_3", "bm25_scores_4", "bm25_scores_5"]])

print(f"MAE: {mean_absolute_error(df_both_test['real_score'], y_pred)}")
print(f"MSE: {mean_squared_error(df_both_test['real_score'], y_pred)}")

MAE: 0.7271214642262895
MSE: 1.259567387687188


C:\Users\courn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [26]:
reg = SVR().fit(df_both[["pipe_score", "pipe_label", "bm25_ratings_1", "bm25_ratings_2", "bm25_ratings_3", "bm25_ratings_4", "bm25_ratings_5", "bm25_scores_1", "bm25_scores_2", "bm25_scores_3", "bm25_scores_4", "bm25_scores_5"]], df_both["real_score"])

y_pred = reg.predict(df_both_test[["pipe_score", "pipe_label", "bm25_ratings_1", "bm25_ratings_2", "bm25_ratings_3", "bm25_ratings_4", "bm25_ratings_5", "bm25_scores_1", "bm25_scores_2", "bm25_scores_3", "bm25_scores_4", "bm25_scores_5"]])

print(f"MAE: {mean_absolute_error(df_both_test['real_score'], y_pred)}")
print(f"MSE: {mean_squared_error(df_both_test['real_score'], y_pred)}")

MAE: 0.911549568336651
MSE: 1.1600571423842934


In [27]:
reg = XGBRegressor().fit(df_both[["pipe_score", "pipe_label", "bm25_ratings_1", "bm25_ratings_2", "bm25_ratings_3", "bm25_ratings_4", "bm25_ratings_5", "bm25_scores_1", "bm25_scores_2", "bm25_scores_3", "bm25_scores_4", "bm25_scores_5"]], df_both["real_score"])

y_pred = reg.predict(df_both_test[["pipe_score", "pipe_label", "bm25_ratings_1", "bm25_ratings_2", "bm25_ratings_3", "bm25_ratings_4", "bm25_ratings_5", "bm25_scores_1", "bm25_scores_2", "bm25_scores_3", "bm25_scores_4", "bm25_scores_5"]])

print(f"MAE: {mean_absolute_error(df_both_test['real_score'], y_pred)}")
print(f"MSE: {mean_squared_error(df_both_test['real_score'], y_pred)}")

MAE: 0.7354411515538982
MSE: 0.8689947146491467


In [28]:
reg = RandomForestRegressor().fit(df_both[["pipe_score", "pipe_label", "bm25_ratings_1", "bm25_ratings_2", "bm25_ratings_3", "bm25_ratings_4", "bm25_ratings_5", "bm25_scores_1", "bm25_scores_2", "bm25_scores_3", "bm25_scores_4", "bm25_scores_5"]], df_both["real_score"])

y_pred = reg.predict(df_both_test[["pipe_score", "pipe_label", "bm25_ratings_1", "bm25_ratings_2", "bm25_ratings_3", "bm25_ratings_4", "bm25_ratings_5", "bm25_scores_1", "bm25_scores_2", "bm25_scores_3", "bm25_scores_4", "bm25_scores_5"]])

print(f"MAE: {mean_absolute_error(df_both_test['real_score'], y_pred)}")
print(f"MSE: {mean_squared_error(df_both_test['real_score'], y_pred)}")

MAE: 0.7139434276206322
MSE: 0.7661128119800333


In [29]:
pickle.dump(reg, open("both_model.pkl", "wb"))

# Main algorithm

In [6]:
train = pd.read_csv("reviews_train.csv")

In [7]:
train["review_text"] = train["review_text"].apply(lambda x: ast.literal_eval(x))

In [8]:
train["review_rating"].value_counts()

review_rating
1    381
2    381
3    381
4    381
5    381
Name: count, dtype: int64

In [9]:
# Liste d'avis
documents = train["review_text"].tolist()

# Listes de scores
ratings = train["review_rating"].tolist()

# Créer un modèle BM25
bm25 = BM25Okapi(documents)


In [10]:
def getTopDocs(bm25, query, documents, ratings, top=5):
    # Calculer les scores de similarité
    scores = bm25.get_scores(query)

    # Associer chaque avis à son score
    doc_scores = list(zip(documents, scores, ratings))

    # Trier les avis par score décroissant
    return sorted(doc_scores, key=lambda x: x[1], reverse=True)[:top]

In [57]:
""" Not used
# Obtenir les mots positifs et négatifs en utilisant bert
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

def getSentiment(text):
    # Tokenize the text
    tokens = tokenizer.encode(text, return_tensors='pt')
    # Reduce the tokens size to 512
    tokens = tokens[:, :512]
    # Get the prediction
    result = model(tokens)
    # Return the label
    return torch.argmax(result.logits).item()
"""

In [58]:
""" Not used
def getRevelantWords(most_freq, pos_nb = 5, neg_nb = 5):
    pos_words = []
    neg_words = []
    for word, nb in most_freq:
        if type(word) == tuple:
            word = " ".join(word)
        if getSentiment(word) == 0 and len(neg_words) < neg_nb:
            neg_words.append(word)
        elif getSentiment(word) == 4 and len(pos_words) < pos_nb:
            pos_words.append(word)

        if len(neg_words) == neg_nb and len(pos_words) == pos_nb:
            break

    return pos_words, neg_words
"""

In [11]:
def separer_phrase(phrase):
    # On ajoute des points aux sauts de ligne
    phrase = phrase.replace('\n', '.')

    # Divise d'abord la phrase en utilisant les points, points d'interrogation, points d'exclamation.
    pattern = r'(?<=[.!?])(?=\s|[A-Z"\'(])'
    groupes = re.split(pattern, phrase)

    groupes_fins = []
    for groupe in groupes:
        # Combinaison des motifs de virgule et "et" en une seule expression régulière
        # Sépare sur les virgules (en évitant les nombres décimaux), sur les ; et : et sur les conjonctions de coordinations (avec un contexte spécifique)
        pattern_combined = r'(?<=.{20},)\s(?!\d)|[;:]|\b(mais|ou|et|donc|or|ni|car)\b(?=.{15,})'
        sous_groupes = re.split(pattern_combined, groupe)
        groupes_fins.extend(sous_groupes)

    return [groupe.strip() for groupe in groupes_fins if groupe is not None and groupe.strip() and len(groupe.strip()) > 6]

In [12]:
BOTH_MODEL = pickle.load(open("both_model.pkl", "rb"))
PIPE_MODEL = pickle.load(open("pipe_model.pkl", "rb"))
BM25_MODEL = pickle.load(open("bm25_model.pkl", "rb"))

In [13]:
def estimate_score(top_docs, origin_query = None, use_bm25 = True, FIABILITY_THRESHOLD = 0.6):
    # Calculer la note moyenne des avis
    if use_bm25 and origin_query is None:
        return BM25_MODEL.predict(np.array([top_docs[i][2] for i in range(len(top_docs))] + [top_docs[i][1] for i in range(len(top_docs))]).reshape(1, -1))
    elif not use_bm25:
        pipe_res = pipe(origin_query)[0]
        return PIPE_MODEL.predict(np.array([pipe_res["score"], 1 if pipe_res["label"] == "POSITIVE" else 0]).reshape(1, -1))
    else:
            pipe_res = pipe(origin_query)[0]
            pipe_score = pipe_res["score"]
            pipe_label = 1 if pipe_res["label"] == "POSITIVE" else 0
            bm25_ratings = [top_docs[i][2] for i in range(len(top_docs))]
            bm25_scores = [top_docs[i][1] for i in range(len(top_docs))]
            return BOTH_MODEL.predict(np.array([pipe_score, pipe_label] + bm25_ratings + bm25_scores).reshape(1, -1))

In [14]:
def getRevelantSentences(origin_query, most_freq, documents, ratings, top=5, use_bm25 = True, use_pipe = True):

    # Appel de la fonction
    groupes = separer_phrase(origin_query)

    # Obtenir les scores de chaque groupe
    scores = []
    for groupe in groupes:
        if use_pipe and use_bm25:
            scores.append(estimate_score(getTopDocs(bm25, text2Token(groupe), documents, ratings), origin_query))
        elif use_pipe:
            scores.append(estimate_score(None, origin_query, use_bm25=False))
        elif use_bm25:
            scores.append(estimate_score(getTopDocs(bm25, text2Token(groupe), documents, ratings)))
            
    pos_list = []
    neg_list = []
    for groupe, score in zip(groupes, scores):
        group_tokens = text2Token(groupe)
        sumFreq = sum([freq for word, freq in most_freq if word in group_tokens])
        if score is None:
            continue
        if score >= 3.5:
            pos_list.append((groupe, sumFreq))
        elif score <= 2.5:
            neg_list.append((groupe, sumFreq))

    pos_list = [sentence[0] for sentence in sorted(pos_list, key=lambda x: x[1], reverse=True)[:top]]
    neg_list = [sentence[0] for sentence in sorted(neg_list, key=lambda x: x[1], reverse=True)[:top]]

    return pos_list, neg_list

In [15]:
def main(origin_query, bm25=bm25, documents=documents, ratings=ratings, spell=spell, use_bm25 = True, use_pipe = True):
    query = text2Token(origin_query)
    top_docs = []
    most_freq = []
    if use_bm25:
        top_docs = getTopDocs(bm25, query, documents, ratings, top=5)
        most_freq = getMostFrequentWords([doc[0] for doc in top_docs], top=50)
    pos_list, neg_list = getRevelantSentences(origin_query, most_freq, documents, ratings, top=5, use_bm25 = True, use_pipe = True)
    if not use_pipe:
        origin_query = None
    return estimate_score(top_docs, origin_query, use_bm25=use_bm25), pos_list, neg_list

# Test Model

## Use data unsed and balanced the new dataset

In [16]:
test = pd.read_csv("reviews_test.csv")

test["review_rating"].value_counts()

review_rating
1    152
3    152
4    152
5    152
Name: count, dtype: int64

## Example on test dataset

In [17]:
elmt = test.iloc[random.randint(0, len(test))]
origin_query = elmt["review_text"]
score, pos_list, neg_list = main(origin_query, use_bm25=True, use_pipe=False)

print(f"Query: {origin_query}")
print(f"Vrai score: {elmt['review_rating']}")

print(f"Score: {score[0]}")
print(f"Séquences positives: {pos_list}")
print(f"Séquences négatives: {neg_list}")

Query: Super conseil rapidité de reprise de mon véhicule et rapidité a avoir un véhicule neuf nous conseillons fortement Aramis
Vrai score: 3
Score: 3.5598636505876087
Séquences positives: ['Super conseil rapidité de reprise de mon véhicule', 'rapidité a avoir un véhicule neuf nous conseillons fortement Aramis']
Séquences négatives: []


# Compute the average error

In [22]:
# For bm25
ratings_val = []
exception_idx = []
for i in tqdm(range(len(test))):
    try:
        elmt = test.iloc[i]
        origin_query = elmt["review_text"]
        score = estimate_score(getTopDocs(bm25, text2Token(origin_query), documents, ratings))
        ratings_val.append(round(score[0]))
    except:
        exception_idx.append(i)
        continue

test_sucess = test.drop(exception_idx)

print(f"MAE: {mean_absolute_error(test_sucess['review_rating'], ratings_val)}")
print(f"MSE: {mean_squared_error(test_sucess['review_rating'], ratings_val)}")

100%|██████████| 608/608 [05:54<00:00,  1.71it/s]

MAE: 0.8651315789473685
MSE: 1.2861842105263157


In [23]:
# For pipe
ratings_val = []
exception_idx = []
for i in tqdm(range(len(test))):
    try:
        elmt = test.iloc[i]
        origin_query = elmt["review_text"]
        score = estimate_score(None, origin_query, use_bm25=False)
        ratings_val.append(round(score[0]))
    except:
        exception_idx.append(i)
        continue

test_sucess = test.drop(exception_idx)

print(f"MAE: {mean_absolute_error(test_sucess['review_rating'], ratings_val)}")
print(f"MSE: {mean_squared_error(test_sucess['review_rating'], ratings_val)}")

100%|██████████| 608/608 [03:58<00:00,  2.55it/s]

MAE: 0.6788685524126455
MSE: 1.021630615640599


In [35]:
# For bm25 + pipe
ratings_val = []
exception_idx = []
for i in tqdm(range(len(test))):
    try:
        elmt = test.iloc[i]
        origin_query = elmt["review_text"]
        score = estimate_score(getTopDocs(bm25, text2Token(origin_query), documents, ratings), origin_query)
        ratings_val.append(round(score[0]))
    except:
        exception_idx.append(i)
        continue

test_sucess = test.drop(exception_idx)

print(f"MAE: {mean_absolute_error(test_sucess['review_rating'], ratings_val)}")
print(f"MSE: {mean_squared_error(test_sucess['review_rating'], ratings_val)}")

100%|██████████| 608/608 [10:01<00:00,  1.01it/s]

MAE: 0.6821963394342762
MSE: 0.8685524126455907


# Get the topic

In [93]:
frequents_vectors = pd.read_csv("wordsmost_frequent_with_vectors.csv")

In [94]:
frequents_vectors["embedding"] = frequents_vectors["embedding"].apply(lambda x: ast.literal_eval(re.sub(r',+', ',', x.replace(" ", ",").replace("\n", ",")).replace("[,", "[").replace(",]", "]")))

In [95]:
euclidean_distances = lambda x, y: sum([abs(x[i] - y[i]) for i in range(len(x))])**1/2

In [108]:
def text2vector(text):
    all_vectors = [frequents_vectors[frequents_vectors["fr"] == word].iloc[0]["embedding"] for word in text2Token(text, stem=False, use_n_grams=False) if word in frequents_vectors["fr"].tolist()]
    return [sum([vector[i] for vector in all_vectors])/len(all_vectors) for i in range(len(all_vectors[0]))]

In [157]:
topic1 = "probleme"
topic2 = "super qualité"

topic1_vector = text2vector(topic1)
topic2_vector = text2vector(topic2)

text = "produit de bonne qualité, je recommande"
text_vector = text2vector(text)

print(f"Topic 1: {euclidean_distances(topic1_vector, text_vector)}")
print(f"Topic 2: {euclidean_distances(topic2_vector, text_vector)}")

Topic 1: 11.621313425
Topic 2: 8.125361499999999


In [156]:
text2 = "panne"
text2_vector = text2vector(text2)

print(f"Topic 1: {euclidean_distances(topic1_vector, text2_vector)}")
print(f"Topic 2: {euclidean_distances(topic2_vector, text2_vector)}")

Topic 1: 9.328016499999999
Topic 2: 12.449311075
